In [2]:
import pandas as pd
import numpy as np

import re
from itertools import chain
from collections import Counter

from google.cloud import bigquery
from google.oauth2 import service_account

from datetime import datetime, timedelta

import requests
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# InsecureRequestWarning 경고 무시
warnings.simplefilter('ignore', InsecureRequestWarning)

In [3]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"

warnings.filterwarnings("ignore")

In [4]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [5]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [6]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [7]:
# 오늘 날짜
today = datetime.today()#.strftime('%Y%m%d')

# 어제 날짜 계산
ytday = datetime.today() - timedelta(days=1)

# 만약 어제, 오늘이 토요일(5) 또는 일요일(6)이라면, 그 전주 금요일로 변경
if ytday.weekday() == 5:  # 토요일
    ytday -= timedelta(days=1)
elif ytday.weekday() == 6:  # 일요일
    ytday -= timedelta(days=2)
if today.weekday() == 5:  # 토요일
    today -= timedelta(days=1)
elif today.weekday() == 6:  # 일요일
    today -= timedelta(days=2)

# 'YYYYMMDD' 형식으로 변환
ytday = ytday.strftime('%Y%m%d')
today = today.strftime('%Y%m%d')

#### 법정동코드

In [8]:
dist_code = pd.read_csv('C:\py_src\midoproject\data/법정동코드 전체자료.csv',encoding='cp949')

In [9]:
dist_code_list = []
for i in range(dist_code['법정동명'].str.split().apply(len).max()):
    dist_code_temp = dist_code[dist_code['법정동명'].str.split().str[i].str[-1].isin(['읍','면','동','리','가','로']).dropna()]
    dist_code_list.append([dist_code_temp['법정동명'].str.split().str[i].dropna().unique()])

emd_code_list = list(chain(*chain(*dist_code_list)))

#### 종합쇼핑몰 납품상세내역

In [10]:
all_shop_df = get_dataframe_from_bigquery('g2b', 'shop_detail_df_all', bigquerykey_path)

In [11]:
all_shop_df_fin = all_shop_df[['납품요구접수일자', '수요기관명', '납품요구건명', '업체명', '단가', '단위', '수량', '금액', '수요기관코드', '수요기관구분', '수요기관지역명','납품요구지청명']]
all_shop_df_fin = all_shop_df_fin.sort_values(['납품요구접수일자'],ascending=False).reset_index(drop=True)

In [12]:
# 특수문자, 숫자, 영어 제거 함수 (연속된 공백을 단일 공백으로 변환 포함)
def clean_text(text):
    text = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', ' ', text)
    text = re.sub('\s+', ' ', text).strip()  # 연속된 공백을 단일 공백으로 변환
    return text

In [13]:
# 사업명 전처리
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명'].apply(clean_text)
# all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.strip()

all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.split().apply(lambda x: [kw for kw in x if len(kw) > 1]) ## 글자수 1 제외
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].apply((' ').join)

In [14]:
# 수요기관 필터링
dist_nm = pd.Series(all_shop_df_fin['수요기관지역명'].str.split().str[1].dropna().unique())
dist_nm = np.where(dist_nm.apply(len)<=2,dist_nm,dist_nm.str[:-1])

#### 교육청 예산서

In [15]:
# # 빅쿼리에서 불러오기
# edu_budget_df = get_dataframe_from_bigquery('edu','edu_budget',bigquerykey_path)
# edu_budget_df.sort_values('시도').reset_index(drop=True)

In [16]:
# 빅쿼리에서 불러오기
edu_budget_df = get_dataframe_from_bigquery('edu','edu_budget_listup',bigquerykey_path)
edu_budget_df.sort_values('시도').reset_index(drop=True)

,시도,시군구,구분,과업명,금액,면적,예산집행
0,강원특별자치도,강릉교육지원청,공립,경포중학교 야구장 시설개선,"10,900,000","9,298",학교
1,강원특별자치도,강원특별자치도교육청,사립,동해광희고등학교 인조잔디운동장 보수,"51,700,000",None,None
2,강원특별자치도,교육부,국립,강원대학교사범대학부설고등학교 인조잔디운동장조성 및 환경개선,"2,442,350,000",None,None
3,강원특별자치도,인제교육지원청,공립,상남중학교 개방형 운동장 조성,"725,302,000","6,507",교육청
4,강원특별자치도,강원특별자치도교육청,공립,강릉고등학교 야구장 시설개선,"45,930,000","8,812",학교
...,...,...,...,...,...,...,...
167,충청남도,부여교육지원청,공립,부여여자중학교 유해운동장 교체,"132,948,000","2,720",교육청
168,충청남도,당진교육지원청,사립,신평중학교 운동장 교체,"7,560,000",None,None
169,충청남도,예산교육지원청,공립,예산여자중학교 유해운동장 교체,"259,105,000","2,900",교육청
170,충청북도,청주교육지원청,공립,석교초등학교 야구부실내연습장증축,"1,421,325,000",None,교육청


In [17]:
# 전처리
edu_budget_df['과업명_re'] = edu_budget_df['과업명'].apply(clean_text)
# edu_budget_df['과업명_re'] = edu_budget_df['과업명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
edu_budget_df['과업명_re'] = edu_budget_df['과업명_re'].str.strip()

In [18]:
# 교육청 기준 지역명, 교육청명 종합쇼핑몰 형식과 통일
# Define the replacements
replacements = {
    '전라북도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남',
    '충청북도': '충북',
    '충청남도': '충남'
}

# Apply the replacements
edu_budget_df['시도_re'] = edu_budget_df['시도'].replace(replacements)
edu_budget_df['시도_re'] = edu_budget_df['시도_re'].str[:2]

edu_budget_df['시군구_re'] = np.where(edu_budget_df['시군구'].str.contains('교육지원청'),
                                   edu_budget_df['시도'] + edu_budget_df['시군구'].str.replace('교육지원청',''), edu_budget_df['시군구']) 

In [19]:
# 학교 관련 키워드 및 데이터 추출
edu_budget_df['교육청_키워드'] = edu_budget_df['과업명'].apply(lambda x: ''.join(re.findall('\\w+초|\\w+초등학교|\\w+중|\\w+중학교|\\w+고|\\w+고등학교|\\w+학교', x)))

edu_budget_df['교육청_키워드2'] = edu_budget_df['교육청_키워드'].str.split('초|중|고|학교').str[0] ## 학교명 추출

# 일반고 외 학교명 전처리
replacements = {
    '여자': '여',
    '공업': '공',
    '상업': '상',
    '농업': '농',
    '과학': '과'
}
edu_budget_df['교육청_키워드2'] = edu_budget_df['교육청_키워드2'].replace(replacements, regex=True)

In [20]:
# 학교명 등 필요키워드 추출
edu_kwd_temp = edu_budget_df['교육청_키워드2'].dropna().unique()
edu_kwd = edu_kwd_temp[edu_kwd_temp!='']

In [21]:
# 해당 필요키워드 추출
edu_budget_df['키워드'] = edu_budget_df['교육청_키워드2'].apply(lambda x: ''.join(re.findall('|'.join(edu_kwd), x))) ## 학교명
edu_budget_df['키워드2'] = edu_budget_df['교육청_키워드'].apply(lambda x: ''.join(re.findall('초|중|고', x)))

#### 교육청 종합쇼핑몰 전처리

In [22]:
all_shop_df_fin['키워드'] = all_shop_df_fin['납품요구건명'].apply(lambda x: ''.join(re.findall('|'.join(edu_kwd), x)))
all_shop_df_fin['키워드2'] = all_shop_df_fin['납품요구건명'].apply(lambda x: ''.join(re.findall('초|중|고', x)))

In [23]:
# 종합쇼핑몰 기준 지역명 형식과 통일
all_shop_df_fin['수요기관지역명_split1'] = all_shop_df_fin['수요기관지역명'].str.split().str[0]

# Define the replacements
replacements = {
    '전라북도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남',
    '충청북도': '충북',
    '충청남도': '충남'
}

# Apply the replacements
all_shop_df_fin['수요기관지역명_split1'] = all_shop_df_fin['수요기관지역명_split1'].replace(replacements)
all_shop_df_fin['수요기관지역명_split1'] = all_shop_df_fin['수요기관지역명_split1'].str[:2]

In [24]:
# 지역명 추출2
all_shop_df_fin['수요기관지역명_split2'] = all_shop_df_fin['수요기관지역명'].str.split().str[1]

all_shop_df_fin['수요기관지역명_split2'] = np.where(all_shop_df_fin['수요기관지역명_split2'].isnull(),
                                                  all_shop_df_fin['수요기관지역명_split1'],
                                                  all_shop_df_fin['수요기관지역명_split2'])

all_shop_df_fin['수요기관지역명_split2'] = np.where(all_shop_df_fin['수요기관지역명_split2'].apply(len)<=2,
                                                  all_shop_df_fin['수요기관지역명_split2'],
                                                  all_shop_df_fin['수요기관지역명_split2'].str[:-1])

dist_sig_pat = '|'.join(all_shop_df_fin['수요기관지역명_split2'].dropna().unique())

In [25]:
# 교육청 지역명 추출
edu_budget_df['시군구_re2'] = edu_budget_df['시군구_re'].apply(lambda x: ' '.join(re.findall(dist_sig_pat, x)))
edu_budget_df['시군구_re2'] = edu_budget_df['시군구_re2'].str.strip()

In [32]:
# 학교 키워드 기준 병합
mapping_merge_edu_df_fin = pd.merge(edu_budget_df[['시도', '시군구', '과업명', '시도_re', '시군구_re', '시군구_re2', '키워드','키워드2']],
                                all_shop_df_fin[['수요기관명', '납품요구건명', '업체명', '수요기관지역명', '납품요구지청명', '수요기관지역명_split1', '수요기관지역명_split2', '키워드','키워드2']],
                                how='left', on=['키워드','키워드2'])
mapping_merge_edu_df_fin = mapping_merge_edu_df_fin[mapping_merge_edu_df_fin['키워드']!=''].drop_duplicates().reset_index(drop=True)
mapping_merge_edu_df_fin = mapping_merge_edu_df_fin[mapping_merge_edu_df_fin['납품요구건명'].notnull()].drop_duplicates().reset_index(drop=True)

In [33]:
# 지역 일치 확인
mapping_merge_edu_df_final = mapping_merge_edu_df_fin[mapping_merge_edu_df_fin['시도_re']==mapping_merge_edu_df_fin['수요기관지역명_split1']].drop_duplicates().reset_index(drop=True)
mapping_merge_edu_df_final

,시도,시군구,과업명,시도_re,시군구_re,시군구_re2,키워드,키워드2,수요기관명,납품요구건명,업체명,수요기관지역명,납품요구지청명,수요기관지역명_split1,수요기관지역명_split2
0,강원특별자치도,강원특별자치도교육청,강릉고등학교 야구장 시설개선,강원,강원특별자치도교육청,,강릉,고,강원특별자치도교육청 강원특별자치도강릉교육지원청,강릉제일고 축구장 인조잔디 교체공사 관급자재(인조잔디) 구입,주식회사 그린에셀,강원특별자치도 강릉시,강원지방조달청,강원,강릉
1,전라남도,진도교육지원청,석교중학교 운동장 보수,전남,전라남도진도,진도,석교,중,전라남도교육청 전라남도진도교육지원청,석교중 운동장 보수공사 관급자재(인조잔디) 구입,나눔스포텍 주식회사,전라남도 진도군,광주지방조달청,전남,진도
2,경상북도,영덕교육지원청,강구중학교(축구부 쉼터 증축),경북,경상북도영덕,영덕,강구,중,경상북도 영덕군 시설체육사업소,강구대게축구장 관중석 하단부 철거공사 관급자재 인조잔디 구입,주식회사 에버그린필드,경상북도 영덕군,대구지방조달청,경북,영덕
3,대구광역시,대구광역시교육청,대구고등학교 야구장 조명시설교체 및 인조잔디 재조성,대구,대구광역시교육청,,대구,고,대구광역시교육청 대구공업고등학교,대구공고 운동장 재조성을 위한 인조잔디(관급자재) 구매,케이앤비준우(주),대구광역시 동구,대구지방조달청,대구,동구


In [35]:
# 오매칭 확인 및 제거
delete_idx = [0,2,3]
mapping_merge_edu_df_final.drop(delete_idx, inplace=True)

In [37]:
edu_budget_df_final = edu_budget_df[~edu_budget_df['과업명'].isin(mapping_merge_edu_df_final['과업명'])].reset_index(drop=True)

In [39]:
save_dataframe_to_bigquery(edu_budget_df_final,'edu','edu_budget_listup',bigquerykey_path)

Data inserted into table edu_budget_listup successfully.
